In [2]:
import os
import json
import argparse
import yaml
import warnings

from utils import load_model, generate_answer, Retrieve_documents
from retriever import Index_Builder

from transformers import AutoTokenizer
import pandas as pd

from secret import api_key
import guidance
warnings.filterwarnings(action='ignore')

from prompts import refine_query_prompt, confidence_prompt, default_prompt, strategy_ext_prompt, answer_prompt


In [18]:
def main(args, verbose_document: bool = False, verbose_strategy: bool = False):
    
    query = args['query']   
    
    print(query)
    structure_program = guidance(refine_query_prompt, query = query)
    res_refine = generate_answer(args['model_name'], api_key, refine_query_prompt, query = query)
    
    refined_query = res_refine['refined_query']

    sparse_tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
    
    raw_reference_list = Retrieve_documents(
                            query = query, 
                            data_dir = args['data_dir'],
                            sparse_tokenizer = sparse_tokenizer,
                            use_content_type = args['content_type'],
                            top_k = args['top_k'], 
                            check_point_dir = args['check_point_dir'],
                            index_save_dir = args['index_save_dir'],
                            retrieve_mode = args['retrieve_mode'] #'sparse'
                            )
                     
    if verbose_document:       
        print(raw_reference_list)                   
           
    # res_confidence = generate_answer(args['model_name'], api_key, confidence_prompt, query = refined_query, 
    #                                 passage = [raw_reference_list[0]]) #has to be a list
    
    # confidence = res_confidence['confidence'].lower()
    
    confidence = 'yes'
    
    if confidence == 'no':
        res_default = generate_answer(args['model_name'], api_key, default_prompt, query = refined_query)
        strategy = 'default'
        answer = res_default['answer']
        
    else:
        
        try:
            res_strategy = generate_answer(args['model_name'], api_key, strategy_ext_prompt, query = refined_query, 
                                        passages = [raw_reference_list[0]]) # has to be a list
            
            strategy = res_strategy['strategy']
            
        except: # Split Passage into two when given passage is too long
            
            try: 
                first_half = raw_reference_list[0]['contents'][:int(len(raw_reference_list[0])*0.5)]
                last_half = raw_reference_list[0]['contents'][int(len(raw_reference_list[0])*0.5):]
            
            except:
                first_half = raw_reference_list[0]['content'][:int(len(raw_reference_list[0])*0.5)]
                last_half = raw_reference_list[0]['content'][int(len(raw_reference_list[0])*0.5):]
            
            
            res_strategy = generate_answer(args['model_name'], api_key, strategy_ext_prompt, query = refined_query, 
                                passages = [first_half]) # has to be a list
            
            strategy = res_strategy['strategy']
            
            # res_strategy_2 = generate_answer(args['model_name'], api_key, strategy_ext_prompt, query = refined_query, 
            #         passages = [last_half]) # has to be a list
            
            # strategy_2 = res_strategy_2['strategy']
            
            # strategy = strategy + '\n' + strategy_2
            
        if verbose_strategy:
            print(strategy)
        
        res_answer = generate_answer(args['model_name'], api_key, answer_prompt, query = refined_query,
                                     strategy = strategy) # has to be a list

        first_answer = res_answer['answer']
        #final_answer = res_answer['final_answer']
                                
        answer = first_answer
        
    save = {'Question': query, 
            'Confidence': confidence,
            'Strategy': strategy,
            'Reference': raw_reference_list,
            'Answer': answer}  
    
    print(save)  
    
    args['index'] = 0
    
    # save json file        
    with open(os.path.join('/Project/src/youtube_output',str(args['index'])+".json"), 'w', encoding='utf-8') as f:
         f.write(json.dumps(
                        (save), 
                        ensure_ascii=False,
                        indent = '\t'
                        )
                 )
        
    return answer

In [19]:
q_list = [
        '호날두와 메시, 둘 중 어느 선수가 더욱 위대한가요?',
        '최근 축구 감독의 전반적인 역량에 대한 의문이 많은데,이러한 의견이 나오는 이유는?',
        '좋은 선수가 우선인가 좋은 전술이 우선인가?',
        '위대한 감독이자 위대한 선수였던 사람이 적은 이유는?',
        '윙어는 강한 발을 쓰는 것과 약한 발을 쓰는 것 중 뭐가 더 효율적인가',
        '어린 선수 중 차세대 축구 스타 10인을 꼽아본다면?',
        '역대 축구사에서 가장 창의적이었던 전술은?',
        '미하일로 무드리크의 발롱도르 수상 연도?',
        '호날두가 메시보다 위대한 이유',
        '메시보다 호날두가 위대한 이유'
        ]

In [20]:
args = yaml.load(open("/Project/src/config/runner_chatgpt.yaml",'r'), Loader=yaml.FullLoader)

In [72]:
args['query']= '역대 축구선수 중 가장 뛰어났던 선수로 11인 스쿼드를 짜줘'

In [73]:
args

{'model_name': 'gpt-4',
 'query_refine_dir': './prompts/GPT/query_refine.txt',
 'confidence_dir': './prompts/GPT/confidence_check.txt',
 'default_dir': './prompts/GPT/default_prompt.txt',
 'strategy_ext_dir': './prompts/GPT/default_prompt.txt',
 'answer_dir': './prompts/GPT/answer_generation_nocheck.txt',
 'data_dir': '../json_data_new',
 'content_type': 'All',
 'top_k': 1,
 'check_point_dir': './2050iter_model.pt',
 'index_save_dir': './index_save_dir/sparse',
 'retrieve_mode': 'sparse',
 'query': '역대 축구선수 중 가장 뛰어났던 선수로 11인 스쿼드를 짜줘',
 'index': 0}

In [74]:
main(args, verbose_document = True, verbose_strategy = True)

역대 축구선수 중 가장 뛰어났던 선수로 11인 스쿼드를 짜줘


100%|██████████| 388296/388296 [00:00<00:00, 843723.72it/s]


./index_save_dir/sparse/bm25_index.pickle
Loading Saved sparse index
Searching Sparse Index...
[{'title': '[442.list] 잉글랜드 대표팀 역대 스쿼드 TOP5, 최고는?', 'url': 'https://sports.news.naver.com/news?oid=411&aid=0000006153', 'contents': '\n\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t[포포투=유지선 객원 기자]축구 종주국 잉글랜드는 국제무대에서 스포트라이트 밖에 있던 적이 많았다. 하지만 화려한 스쿼드를 자랑하며 전 세계를 긴장시킨 시절도 있었다. 물론 항상 좋았던 건 아니다. 국제무대에서 번번이 쓴맛을 볼 때도 있었지만, 특정 시기에는 황금세대라 불리며 재능 있는 선수들이 쏟아지기도 했다. 과연 잉글랜드 축구 역사상 어느 팀이 가장 뛰어났을까. 화려한 스쿼드를 자랑했던 역대 잉글랜드 대표팀 TOP5를 꼽았다.  5위 2018년-현재대표 스쿼드(3-5-2): 픽포드(GK) - 맥과이어, 스톤스, 워커 - 영, 알리, 헨더슨, 린가드, 트리피어 - 스털링, 케인‘2018 러시아 월드컵’ 준결승전에서 키에런 트리피어가 찬 프리킥이 골망 구석을 흔들었을 때, 30분 정도는 6천만 영국인 모두가 믿었다. 가레스 사우스게이트 감독의 팀이 오픈 플레이에서 찬스를 만들 수 없다는 것을 잊어라. 쉬운 루트로 준결승에 올랐다는 사실을 잊어라. 돌이켜보면 3백이 아주 독특하게 구성돼있다는 것을 잊어라... 이 팀은 모두를 믿게 만들었다.좋은 팀에 필요한 건 두 명의 훌륭한 공격수인데, 사우스게이트 감독은 라힘 스털링과 해리 케인이라는 두 명의 세계적인 공격 옵션을 가지고 있었다. 케인의 경우 세계적으로 가장 많은 페널티킥을 얻어내는 선수다. 백업에는 마커스 래시포드가 있었다. 이 팀은 잘 훈련돼 있었고, 성실하며 가장 인상적이었다. 경기에서는 침착하기까지 했다. 여기에 메이슨 그

'전략에 따라 답변을 제공하려면, 먼저 역대 잉글랜드 대표팀의 스쿼드를 확인하고, 그 중에서 가장 뛰어난 선수들을 선별해야 합니다. 그러나 제공된 전략에는 구체적인 선수 이름이나 세부 정보가 없으므로, 제 개인적인 지식을 바탕으로 답변을 제공하겠습니다.\n\n골키퍼: 골든 블록을 수차례 세운 레전드 골키퍼인 고든 뱅크스를 선정합니다.\n\n수비수: 보비 무어, 테리 부트쳐, 애쉴리 콜, 게리 네빌을 선정합니다. 보비 무어는 잉글랜드 대표팀이 월드컵에서 우승한 1966년 대표팀의 주장이었습니다.\n\n미드필더: 미드필더 포지션에는 폴 스콜스, 데이비드 베컴, 스티븐 제라드를 선정합니다. 이 선수들은 각각 맨체스터 유나이티드, 레알 마드리드, 리버풀에서 활약하며 세계적인 명성을 얻었습니다.\n\n공격수: 공격수 포지션에는 웨인 루니와 게리 라인커를 선정합니다. 웨인 루니는 잉글랜드 대표팀의 역대 최다 득점자이며, 게리 라인커는 1986년 월드컵에서 황금신발을 수상한 선수입니다.\n\n이렇게 선정된 선수들은 각자의 포지션에서 뛰어난 능력과 업적을 가지고 있으며, 잉글랜드 축구의 역사를 대표하는 인물들입니다. 이들을 기준으로 11인 스쿼드를 구성하면, 역대 가장 뛰어난 선수들로 이루어진 팀을 만들 수 있을 것입니다.'